In [0]:
!tar zxvf ./drive/My\ Drive/Colab\ Notebooks/train.tgz

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install wandb
!wandb login

# CAPTCHA Recognition - 6 digits

In [0]:
import tensorflow.python.keras.applications.densenet as densenet

In [0]:
from tensorflow.keras.layers import  Dense

In [0]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
from tensorflow.python.keras import backend
from tensorflow.python.keras import layers
from tensorflow.python.keras.applications import imagenet_utils
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.utils import data_utils
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export

In [0]:
import wandb

# Parsing Data

In [0]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    return img

In [0]:
#training data 01 
train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
data = pd.read_csv("./train/data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

for i in range(6):
    data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
train_data = train_dir.map(lambda x: readimg_to_tensor(x))
train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data01=tf.data.Dataset.zip((train_data,train_label))

In [0]:
#training data 02
train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
data2 = pd.read_csv("./train/data02_train.csv")
for i in range(len(data2)):
    data2.iloc[i,1] = list(data2.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data2)):
        arr[j][i] = data2.iloc[i,1][j]
data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

#to one hot
for i in range(6):
    data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data02=tf.data.Dataset.zip((train_data2,train_label2))

In [0]:
#number of element
num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements

50000

In [0]:
def prepData01(): 
  validation_split=0.1
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(50000).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train,lb_train,img_test,lb_test

# Model configeration

In [0]:
def conv_block(x, growth_rate, name):
  """A building block for a dense block.
  Arguments:
    x: input tensor.
    growth_rate: float, growth rate at dense layers.
    name: string, block label.
  Returns:
    Output tensor for the block.
  """
  bn_axis = 3 
  x1 = layers.BatchNormalization(
      axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(x)
  x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
  x1 = layers.Conv2D(
      4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(
          x1)
  x1 = layers.BatchNormalization(
      axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(
          x1)
  x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
  x1 = layers.Conv2D(
      growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(
          x1)
  
  x1 = layers.Dropout(0.3)(x1)
  x = layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
  return x

def dense_block(x, blocks, name):
  """A dense block.
  Arguments:
    x: input tensor.
    blocks: integer, the number of building blocks.
    name: string, block label.
  Returns:
    Output tensor for the block.
  """
  for i in range(blocks):
    x = conv_block(x, 32, name=name + '_block' + str(i + 1))
  return x

In [0]:
def transition_block(x, reduction, name):
  """A transition block.
  Arguments:
    x: input tensor.
    reduction: float, compression rate at transition layers.
    name: string, block label.
  Returns:
    output tensor for the block.
  """
  bn_axis = 3 
  x = layers.BatchNormalization(
      axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(
          x)
  x = layers.Activation('relu', name=name + '_relu')(x)
  x = layers.Conv2D(
      int(backend.int_shape(x)[bn_axis] * reduction),
      1,
      use_bias=False,
      name=name + '_conv')(
          x)
  x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
  return x

In [0]:
def buildModel(theblocks,dropoutrate=0.5,modelname="myDenseNet"):  
  bn_axis = 3
  img_input = layers.Input(shape=(60,200,3))
  blocks = theblocks #[6,6,24,16]

  x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
  #x = layers.Dropout(0.1)(x)
  x = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(x)
  x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(
            x)
  x = layers.Activation('relu', name='conv1/relu')(x)
  x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
  x = layers.MaxPooling2D(3, strides=2, name='pool1')(x)
  x = layers.Dropout(dropoutrate)(x)

  x = dense_block(x, blocks[0], name='conv2')
  x = transition_block(x, 0.5, name='pool2')
  x = layers.Dropout(dropoutrate)(x)

  x = dense_block(x, blocks[1], name='conv3')
  x = transition_block(x, 0.5, name='pool3')
  x = layers.Dropout(dropoutrate)(x)

  x = dense_block(x, blocks[2], name='conv4')
  x = transition_block(x, 0.5, name='pool4')
  x = layers.Dropout(dropoutrate)(x)

  x = dense_block(x, blocks[3], name='conv5')

  x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
  x = layers.Activation('relu', name='relu')(x)

  x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
  digit1 = Dense(36, name='digit1', activation='softmax')(x)
  digit2 = Dense(36, name='digit2', activation='softmax')(x)
  digit3 = Dense(36, name='digit3', activation='softmax')(x)
  digit4 = Dense(36, name='digit4', activation='softmax')(x)
  digit5 = Dense(36, name='digit5', activation='softmax')(x)
  digit6 = Dense(36, name='digit6', activation='softmax')(x)
  modelt = tf.keras.models.Model(inputs=img_input,outputs = [digit1 , digit2, digit3, digit4, digit5, digit6], name = modelname)
  modelt.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9), 
    metrics=['accuracy'])
  return modelt

In [0]:
def trainModel(model,img_train,lb_train,img_test,lb_test,callB):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = 128,
        shuffle = True,
        validation_data=(img_test, lb_test),
        epochs=100,
        verbose=1,
        callbacks = callB
        )

In [0]:
modelt = buildModel([6,6,24,16])
modelt.summary()

In [0]:
runcount=0
projectname = "my-project-2"
filename="weights.6_6_24_16_g32"
img_train,lb_train,img_test,lb_test = prepData01()

In [0]:
blocks = [6,6,24,16]
wandb.init(project=projectname,name = "20200526_good_run_"+str(runcount),reinit = True)
filepath=filename+str(runcount)+".hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,mode='min')
mymodel = buildModel(blocks,modelname='myDenseNet'+str(runcount))
callbacks_list = [checkpoint, wandb.keras.WandbCallback()]
trainModel(mymodel,img_train,lb_train,img_test,lb_test,callbacks_list)
runcount+=1

In [0]:

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,
mode='min')
callbacks_list = [checkpoint, wandb.keras.WandbCallback()]